In [34]:
import pandas as pd
from datetime import datetime
sphist = pd.read_csv('YAHOO-INDEX_GSPC.csv')
sphist.head()

,Date,Open,High,Low,Close,Volume,Adjusted Close
0,2016-07-08,2106.969971,2131.709961,2106.969971,2129.899902,3.607500e+09,2129.899902
1,2016-07-07,2100.419922,2109.080078,2089.389893,2097.899902,3.604550e+09,2097.899902
2,2016-07-06,2084.429932,2100.719971,2074.020020,2099.729980,3.909380e+09,2099.729980
3,2016-07-05,2095.050049,2095.050049,2080.860107,2088.550049,3.658380e+09,2088.550049
4,2016-07-01,2099.340088,2108.709961,2097.899902,2102.949951,3.458890e+09,2102.949951


In [63]:
sphist['Date'] =pd.to_datetime(sphist['Date'])
new_sphist = sphist[sphist['Date']>datetime(year =2015, month = 4,day = 1)]
new_sphist.shape

(320, 12)

In [28]:
new_sphist.sort_values('Date', ascending = True,inplace = True)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [60]:
sphist.sort_values('Date', ascending = True, inplace = True)
day_mean = sphist['Close'].rolling(window=5).mean()
sphist['5_day_mean'] = day_mean.shift(1)
sphist['30_day_mean'] = sphist['Close'].rolling(window = 30).mean().shift(1)
sphist['30_day_std'] = sphist['Close'].rolling(window = 30).std().shift(1)
sphist['365_day_mean'] = sphist['Close'].rolling(window = 365).mean().shift(1)
sphist['365_day_std'] = sphist['Close'].rolling(window = 365).std().shift(1)
sphist.dropna(axis = 0,inplace = True)

In [75]:
train = sphist[sphist['Date']<datetime(year = 2013, month = 1, day = 1)]
test = sphist[sphist['Date']>=datetime(year = 2013, month =1, day =1)]
train.head()


,Date,Open,High,Low,Close,Volume,Adjusted Close,5_day_mean,30_day_mean,30_day_std,365_day_std,365_day_mean
16371,1951-06-19,22.020000,22.020000,22.020000,22.020000,1100000.0,22.020000,21.800,21.703333,0.473595,1.790253,19.447726
16370,1951-06-20,21.910000,21.910000,21.910000,21.910000,1120000.0,21.910000,21.900,21.683000,0.444648,1.789307,19.462411
16369,1951-06-21,21.780001,21.780001,21.780001,21.780001,1100000.0,21.780001,21.972,21.659667,0.411452,1.788613,19.476274
16368,1951-06-22,21.549999,21.549999,21.549999,21.549999,1340000.0,21.549999,21.960,21.631000,0.368514,1.787659,19.489562
16367,1951-06-25,21.290001,21.290001,21.290001,21.290001,2440000.0,21.290001,21.862,21.599000,0.329130,1.786038,19.502082


In [85]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
train_data = ss.fit_transform(train[['5_day_mean','30_day_mean','30_day_std','365_day_mean','365_day_std']])
test_data = ss.transform(test[['5_day_mean','30_day_mean','30_day_std','365_day_mean','365_day_std']])

In [87]:
lr = LinearRegression()
lr.fit(train_data,train['Close'])
close_pred = lr.predict(test_data)
lr.residues_

//anaconda/lib/python3.5/site-packages/sklearn/utils/__init__.py:93: DeprecationWarning: Function residues_ is deprecated; ``residues_`` is deprecated and will be removed in 0.19
  warnings.warn(msg, category=DeprecationWarning)


1683117.8383145721

In [92]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
mean_absolute_error = mean_absolute_error(y_true = test['Close'],y_pred = close_pred)
mean_squared_error = mean_squared_error(y_true = test['Close'],y_pred = close_pred)
mean_squared_error

537.51448287174662

In [94]:
test['close_pred'] = close_pred

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [95]:
test.head()

,Date,Open,High,Low,Close,Volume,Adjusted Close,5_day_mean,30_day_mean,30_day_std,365_day_std,365_day_mean,close_pred
885,2013-01-02,1426.189941,1462.430054,1426.189941,1462.420044,4.202600e+09,1462.420044,1418.641992,1414.258667,17.834740,90.463948,1327.534055,1418.586176
884,2013-01-03,1462.420044,1465.469971,1455.530029,1459.369995,3.829730e+09,1459.369995,1425.793994,1417.676668,16.852563,90.738976,1327.908247,1425.997715
883,2013-01-04,1459.369995,1467.939941,1458.989990,1466.469971,3.424290e+09,1466.469971,1433.702002,1420.092668,17.470824,90.995857,1328.224877,1434.401585
882,2013-01-07,1466.469971,1466.469971,1456.619995,1461.890015,3.304970e+09,1461.890015,1443.376001,1422.714665,18.339803,91.279049,1328.557617,1444.715285
881,2013-01-08,1461.890015,1461.890015,1451.640015,1457.150024,3.601600e+09,1457.150024,1455.267993,1425.076664,18.678333,91.544368,1328.898603,1457.418324


In [97]:
lr.score(test_data,test['Close'])

0.98452551671364741